In [1]:
import numpy as np
import pandas as pd
import chardet
import urllib

In [2]:
CSV_FILE_PATH = './HW1data.csv'
with open(CSV_FILE_PATH,'rb') as data:
    result = chardet.detect(data.read(100000))
data_encoding=result['encoding']

In [3]:
df = pd.read_csv(CSV_FILE_PATH,encoding=data_encoding)
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,4.838649e+14,5751504.0,F,2016-05-31T10:14:47Z,2016-06-08T00:00:00Z,21.0,ILHA DE SANTA MARIA,0.0,0.0,0.0,0.0,0.0,1.0,No
1,4.858271e+13,5736107.0,F,2016-05-25T07:55:55Z,2016-05-25T00:00:00Z,36.0,ANDORINHAS,1.0,0.0,0.0,0.0,0.0,0.0,No
2,7.156662e+12,5708125.0,F,2016-05-17T10:39:38Z,2016-06-06T00:00:00Z,41.0,ITARARÉ,0.0,0.0,0.0,0.0,0.0,0.0,No
3,8.816117e+14,6313735.0,NaN,2017-03-30T05:52:45Z,2017-02-08T00:00:00Z,27.0,TABUAZEIRO,0.0,1.0,0.0,0.0,0.0,0.0,No
4,1.158164e+14,6261424.0,F,2016-12-16T23:34:44Z,2016-11-16T00:00:00Z,NaN,PRAIA DO SUÁ,0.0,0.0,0.0,1.0,0.0,0.0,No


In [4]:
missing_values_count = df.isnull().sum()
row_num = df.shape[0]
missing_values_percentage = np.round(missing_values_count[df.columns]/row_num*100,2)
print("Percentage of missing data columns: ")
print(missing_values_percentage)

total_cells = np.product(df.shape)
total_missing = missing_values_count.sum()
print("Percentage of total missing data: ", np.round(total_missing/total_cells*100,2), "%")

Percentage of missing data columns: 
PatientId         9.50
AppointmentID     9.53
Gender            9.59
ScheduledDay      9.59
AppointmentDay    9.57
Age               9.54
Neighbourhood     9.53
Scholarship       9.60
Hipertension      9.56
Diabetes          9.54
Alcoholism        9.57
Handcap           9.55
SMS_received      9.57
No-show           9.55
dtype: float64
Percentage of total missing data:  9.56 %


In [120]:
df_fillna = df.copy()
key = 'PatientId'
subset = ['Gender','Age','Neighbourhood']
for column in subset:
    df_fillna[column+'_new'] = df_fillna.groupby(key)[column].transform('first')
    df_fillna[column].fillna(df_fillna[column+'_new'],inplace=True)
    df_fillna.drop([column+'_new'],axis=1,inplace=True)


In [96]:
missing_values_count = df_fillna.isnull().sum()
row_num = df_fillna.shape[0]
key_subset=subset.copy()
key_subset.insert(0,key)
missing_values_percentage = np.round(missing_values_count[key_subset]/row_num*100,2)
print("Percentage of missing data columns: ")
print(missing_values_percentage)


Percentage of missing data columns: 
PatientId        9.50
Gender           1.10
Age              1.08
Neighbourhood    1.08
dtype: float64


In [101]:
m=df[df.PatientId.isnull()]
m[df[subset].notnull().T.all()][subset]

True

In [106]:
df_fillna[df_fillna.PatientId.isnull() & df_fillna[subset].notnull().T.all()][key_subset]


,PatientId,Gender,Age,Neighbourhood
24,NaN,F,55.0,SANTOS DUMONT
39,NaN,F,59.0,FONTE GRANDE
42,NaN,F,35.0,GRANDE VITÓRIA
43,NaN,M,39.0,TABUAZEIRO
80,NaN,M,59.0,RESISTÊNCIA
...,...,...,...,...
667459,NaN,F,69.0,BONFIM
667482,NaN,F,43.0,ITARARÉ
667485,NaN,F,29.0,MARUÍPE
667519,NaN,F,34.0,MARIA ORTIZ


In [105]:
df_fillna[key+'_new'] = df_fillna.groupby(subset)[key].transform('first')
df_fillna[df_fillna[key+'_new'].notnull()&df_fillna[key].isnull()]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,PatientId_new
24,NaN,7215198.0,F,2017-04-24T14:52:50Z,2016-05-26T00:00:00Z,55.0,SANTOS DUMONT,0.0,1.0,0.0,0.0,4.0,0.0,No,8.615864e+11
39,NaN,6947275.0,F,NaN,2016-08-18T00:00:00Z,59.0,FONTE GRANDE,0.0,0.0,1.0,0.0,2.0,1.0,No,3.272469e+12
42,NaN,6721383.0,F,2017-05-03T18:50:23Z,2017-04-30T00:00:00Z,35.0,GRANDE VITÓRIA,0.0,1.0,0.0,0.0,2.0,0.0,Yes,4.527948e+13
43,NaN,7627886.0,M,2016-09-14T21:39:01Z,2016-07-25T00:00:00Z,39.0,TABUAZEIRO,0.0,0.0,0.0,1.0,1.0,1.0,No,6.626197e+14
80,NaN,6925609.0,M,2017-04-24T17:20:15Z,2016-08-19T00:00:00Z,59.0,RESISTÊNCIA,0.0,0.0,0.0,0.0,4.0,0.0,No,1.872838e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667459,NaN,7488291.0,F,2017-04-28T15:10:43Z,2016-05-20T00:00:00Z,69.0,BONFIM,0.0,0.0,NaN,0.0,0.0,1.0,No,3.792467e+12
667482,NaN,6774538.0,F,2017-05-07T01:29:04Z,2017-03-06T00:00:00Z,43.0,ITARARÉ,0.0,0.0,0.0,0.0,0.0,0.0,No,2.592888e+14
667485,NaN,6911478.0,F,NaN,2016-07-01T00:00:00Z,29.0,MARUÍPE,0.0,0.0,0.0,0.0,0.0,1.0,No,7.242365e+12
667519,NaN,NaN,F,2016-10-24T03:03:04Z,2016-09-22T00:00:00Z,34.0,MARIA ORTIZ,0.0,0.0,1.0,1.0,0.0,0.0,No,4.158235e+14


In [121]:
m=df_fillna[df_fillna.PatientId.notnull()].drop_duplicates(key_subset)
m.shape

(63467, 14)

In [126]:
# m.drop_duplicates(key).shape
x=(int)(m[m.PatientId.duplicated()][0:1].PatientId)
t=m[m.PatientId.duplicated()][0:1].ScheduledDay

In [123]:
m[m.PatientId==x]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
322,3.494352e+14,7681811.0,M,2017-05-22T08:10:16Z,NaN,45.0,DA PENHA,0.0,0.0,0.0,0.0,0.0,1.0,No
1555,3.494352e+14,5712457.0,M,2016-05-18T08:54:15Z,2016-06-01T00:00:00Z,46.0,DA PENHA,0.0,0.0,0.0,0.0,0.0,1.0,No


In [135]:
a=df.PatientId.unique()
np.NAN in a

False

In [147]:
test = pd.DataFrame({'a': [1, 1, np.NaN, 2, 3],
                  'b': [1, 2, 3, 4, 5],
                  'c': np.random.randn(5)})

m=test['a'].unique()
np.isnan(m)

True